Comparing the compute time for running the interior-point method on three different formulations of the optimization problem: (1) dual, (2) primal, and (3) modified primal.

In [1]:
using RCall
using Mosek
using JuMP
using PyPlot
using Distributions
using LowRankApprox
using DataFrames
using CVS

LoadError: [91mArgumentError: Module CVS not found in current path.
Run `Pkg.add("CVS")` to install the CVS package.[39m

In [2]:
include("../code/julia/datasim.jl");
include("../code/julia/likelihood.jl");
include("../code/julia/mixSQP_time.jl");
include("../code/julia/primaldual.jl");
include("../code/julia/REBayes.jl");

In [3]:
srand(1);
n = round(Int,1e4); m = 40;
x = normtmixdatasim(n);
sd = autoselectmixsd(x,nv = m);
L  = normlikmatrix(x,sd = sd);
mixSQP_time(L);

In [4]:
a = zeros(10,7);

In [12]:
a

10×7 Array{Float64,2}:
 4.29507     0.715262   0.124215   0.778613  0.309352   0.00503396  0.003
 0.00745568  0.0744612  0.0555333  0.522175  0.0161516  0.00282402  0.005
 0.0229443   0.127992   0.127429   0.976991  0.0362826  0.00269214  0.01 
 0.0343323   0.254678   0.255791   1.75815   0.0377954  0.00616137  0.016
 0.0720909   0.509059   0.517084   3.34384   0.0702588  0.00739223  0.031
 0.219612    1.25604    1.31076    8.09865   0.394399   0.0120535   0.073
 0.373561    6.65277    7.40581    0.0       0.41342    0.0273739   0.161
 0.0         0.0        0.0        0.0       0.175641   0.019743    0.319
 0.0         0.0        0.0        0.0       0.466379   0.0438104   0.697
 0.0         0.0        0.0        0.0       1.13479    0.0899928   1.894

In [8]:
# need to repeat this 10 times to get a reliable result.
for i = 1:10
    n = round(Int,40 * 2^i); m = 40;
    x = normtmixdatasim(n);
    sd = autoselectmixsd(x,nv = m);
    L  = normlikmatrix(x,sd = sd);
    #tic(); ip_dual(L); a[i,1] = toc();
    #tic(); ip_simp(L); a[i,2] = toc();
    #tic(); ip_box(L); a[i,3] = toc();
    #tic(); sqp_dual(L); a[i,4] = toc();
    tic(); sqp_simp(L); a[i,5] = toc();
    tic(); sqp_box(L); a[i,6] = toc();
    tic(); a[i,7] = REBayes(L)[2]; toc();
    println(i)
end

elapsed time: 0.309351913 seconds
elapsed time: 0.00503396 seconds
elapsed time: 0.067897332 seconds
1
elapsed time: 0.016151567 seconds
elapsed time: 0.002824022 seconds
elapsed time: 0.062945986 seconds
2
elapsed time: 0.036282599 seconds
elapsed time: 0.002692144 seconds
elapsed time: 0.06791897 seconds
3
elapsed time: 0.037795367 seconds
elapsed time: 0.006161372 seconds
elapsed time: 0.074754031 seconds
4
elapsed time: 0.070258847 seconds
elapsed time: 0.007392226 seconds
elapsed time: 0.090313865 seconds
5
elapsed time: 0.39439885 seconds
elapsed time: 0.01205353 seconds
elapsed time: 0.135370463 seconds
6
elapsed time: 0.413419538 seconds
elapsed time: 0.027373895 seconds
elapsed time: 0.223580379 seconds
7
elapsed time: 0.175640866 seconds
elapsed time: 0.019742951 seconds
elapsed time: 0.380833743 seconds
8
elapsed time: 0.466378736 seconds
elapsed time: 0.043810356 seconds
elapsed time: 0.764182441 seconds
9
elapsed time: 1.134785181 seconds
elapsed time: 0.089992815 seconds


In [11]:
#plot(log.(a)); legend(["ip_dual";"ip_simp";"ip_box";"sqp_dual";"sqp_simp";"sqp_box";"REBayes"])

In [10]:
out = [["n" "ip_dual" "ip_simp" "ip_box" "sqp_dual" "sqp_simp" "sqp_box" "REBayes"]; [1;2;3;4;5;6;7;8;9;10]*80 a];

In [152]:
@rput a
R"write.csv(a,'out.txt')"

RCall.RObject{RCall.NilSxp}
NULL
